In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import os
import sys
sys.path.append('../')
from compute_metrics import Model
import json
from lib.TabularDataset import dataset_params
import copy
from sklearn.metrics import roc_auc_score
import hashlib

res_dir = Path('/scratch/hdd001/home/haoran/explanations-subpopulations/')

In [21]:
df_raw = []
for path in res_dir.glob('**/*.csv'):
    args = json.load((path.parent/'args.json').open('r'))
    hparams = [i for i in list(args.keys()) if i not in  ['seed', 'output_dir']]
    args['hparam_id'] = hashlib.md5(str([args[i] for i in hparams]).encode('utf-8')).hexdigest()
    res = pd.read_csv(path)
    args['accuracy_all'] = ((res['expl_pred'].values > 0.5) == res[
                                  'blackbox_pred']).sum() / len(res)
    try:
        args['auroc_all'] = roc_auc_score(res['blackbox_pred'].values, res['expl_pred'].values)
    except ValueError:
        # print(args)
        args['auroc_all'] = 0.5
    
    group_names = dataset_params[args['dataset']].sensitive_attributes
    unique_groups = res[group_names].drop_duplicates()
    for grp in group_names:
        for val in res[grp].unique():
            unique_groups = unique_groups.append({grp: val, **{i: np.nan for i in group_names if i != grp}}, ignore_index = True)
    
    for group in range(unique_groups.shape[0]):
        group_i = unique_groups.iloc[group].values
        mask = ~pd.isnull(group_i)
        sel_rows = res[(res[np.array(group_names)[mask]].values ==
                       group_i[mask]).all(1)]

        group_0_val_model = Model(sel_rows['expl_pred'].values, sel_rows[
                                  'blackbox_pred'].values)
        all_metrics = group_0_val_model.compute()
        args_group = {
            **copy.deepcopy(args),
            **all_metrics
                     }
        
        args_group['group'] = str(unique_groups.iloc[group].values)
        args_group['n'] = len(sel_rows)
        args_group['level'] = mask.sum()
        
        df_raw.append(args_group)

df = pd.DataFrame(df_raw)

In [24]:
def agg_func(x):
    res = {}
    for met, disp in zip(['ACC', 'AUROC'], ['accuracy', 'auroc']):
        res[f'{disp}_all'] = x[f'{disp}_all'].iloc[0]
        res[f'{disp}_min'] = x[met].min()
        res[f'{disp}_minority'] = x.loc[x['n'].idxmin(), met]
        res[f'worst_group_{disp}'] = x.loc[x[met].idxmin(), 'group']
        
    return pd.Series(res)

def bold_max(s):
    fmts = []
    for grp in s.index.get_level_values(0).unique():
        max_seq = [i for i in s[grp] if isinstance(i, (float, np.float32, np.float64))]
        max_val = max(max_seq) if len(max_seq) else 0
        fmts += ['font-weight: bold' if i == max_val else '' for i in s[grp]]
    return fmts

In [35]:
LEVEL = 2

res_df = df.query(f'level == {LEVEL}').groupby('output_dir').apply(agg_func)
metrics = [i for i in res_df.columns if not i.startswith('worst_group')]
out1 = (res_df
     .reset_index()
     .merge(df[hparams + ['output_dir', 'seed', 'hparam_id']].drop_duplicates()))

out2 = (out1.groupby('hparam_id').agg({
   i: ('mean', 'std') for i in metrics
})
       .merge(df[hparams + ['hparam_id']].drop_duplicates().set_index('hparam_id'), left_index = True, right_index = True)
       .reset_index())

for col in metrics:
    out2[col] = out2[(col, 'mean')].apply(lambda x: '{0:.1%}'.format(x)) +' ± ' +  out2[(col, 'std')].apply(lambda x: '{0:.1%}'.format(x))

out2.pivot_table(values = metrics, 
                  index = ['dataset', 'blackbox_model', 'n_features'], columns = ['model_type'], aggfunc = lambda x: x)

/h/haoran/anaconda3/lib/python3.6/site-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)


accuracy_all                \
model_type                                     ARL           ERM   
dataset    blackbox_model n_features                               
adult      lr             5           95.5% ± 0.1%  93.1% ± 0.5%   
                          10          80.0% ± 0.9%  54.7% ± 0.6%   
           nn             5           69.8% ± 7.1%  72.4% ± 5.0%   
                          10          62.2% ± 4.0%  64.3% ± 2.0%   
           rf             5           92.8% ± 0.5%  93.1% ± 0.5%   
                          10          79.5% ± 0.4%  71.9% ± 0.3%   
           svm_rbf        5           90.7% ± 0.3%  91.4% ± 0.2%   
                          10          84.5% ± 0.1%  80.8% ± 0.4%   
           xgb            5           94.0% ± 0.1%  93.5% ± 0.1%   
                          10          92.6% ± 0.1%  86.8% ± 0.5%   
recidivism lr             5           92.0% ± 0.7%  89.1% ± 1.1%   
                          10          71.2% ± 2.6%  51.6% ± 1.5%   
           nn             5           78.5% ± 2.5%  77.7% ± 2.1%   
                          10          63.0% ± 4.8%  66.9% ± 3.1%   
           rf             5           78.3% ± 2.1%  80.2% ± 1.2%   
                          10          78.0% ± 1.8%  73.9% ± 0.7%   
           svm_rbf        5           82.9% ± 0.4%  84.5% ± 0.4%   
                          10          73.9% ± 1.3%  72.1% ± 1.2%   
           xgb            5           65.9% ± 1.3%  71.3% ± 1.0%   
                          10          68.4% ± 1.1%  73.8% ± 0.9%   

                                                      accuracy_min  \
model_type                                  sklearn            ARL   
dataset    blackbox_model n_features                                 
adult      lr             5           100.0% ± 0.0%   88.6% ± 2.6%   
                          10          100.0% ± 0.0%  54.7% ± 20.2%   
           nn             5            75.1% ± 7.3%   58.2% ± 9.6%   
                          10           72.7% ± 6.8%   47.4% ± 5.0%   
           rf             5            95.8% ± 0.1%   90.0% ± 1.4%   
                          10           95.4% ± 0.1%   64.1% ± 5.3%   
           svm_rbf        5            93.6% ± 0.1%   86.5% ± 0.6%   
                          10           91.1% ± 0.1%   77.3% ± 1.3%   
           xgb            5            95.5% ± 0.0%   90.7% ± 2.8%   
                          10           95.3% ± 0.1%   87.6% ± 1.1%   
recidivism lr             5           100.0% ± 0.0%   80.6% ± 3.1%   
                          10          100.0% ± 0.0%   65.1% ± 6.8%   
           nn             5            84.1% ± 0.9%   75.9% ± 3.6%   
                          10           79.9% ± 2.7%   56.5% ± 5.8%   
           rf             5            90.4% ± 0.6%   76.6% ± 2.2%   
                          10           88.6% ± 1.9%   76.0% ± 2.1%   
           svm_rbf        5            87.5% ± 0.8%   78.9% ± 0.2%   
                          10           86.1% ± 0.3%   67.2% ± 1.8%   
           xgb            5            77.5% ± 0.5%   59.8% ± 1.5%   
                          10           72.1% ± 0.6%   63.7% ± 1.4%   

                                                                   \
model_type                                     ERM        sklearn   
dataset    blackbox_model n_features                                
adult      lr             5           77.5% ± 6.1%  100.0% ± 0.0%   
                          10          19.7% ± 7.3%  100.0% ± 0.0%   
           nn             5           61.6% ± 4.7%  64.8% ± 10.5%   
                          10          51.6% ± 3.5%  59.2% ± 10.3%   
           rf             5           89.8% ± 2.4%   93.0% ± 0.0%   
                          10          39.5% ± 6.3%   93.0% ± 0.0%   
           svm_rbf        5           85.7% ± 2.8%   91.0% ± 0.2%   
                          10          63.1% ± 2.4%   86.9% ± 0.2%   
           xgb            5           90.3% ± 2.6%   93.0% ± 0.0%   
                          10      